In [1]:
import json
import string
import random 
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
import speech_recognition as sr



In [2]:
# used a dictionary to represent an intents JSON file
with open("../training_data/intents.json", "r", encoding='utf-8') as f:
    data = json.load(f)
    
print(data)

{'intents': [{'tag': 'greetings', 'patterns': ['Hi', 'Hey', 'Hello', 'Good day'], 'responses': ["Hello, I'm the University Chatbot"], 'context_set': 'hwru?'}, {'tag': 'howami', 'patterns': ['How are you?', 'How are you feeling?', "What's up?"], 'responses': ["I'm feeling great as usual, how about you?"], 'context_filter': 'hwru?', 'context_set': 'question'}, {'tag': 'imgood', 'patterns': ["I'm good", "I'm okay", "I'm feeling great as well"], 'responses': ["That's good! So how can I help you today?"], 'context_filter': 'question'}, {'tag': 'imnot', 'patterns': ['Feeling bad', 'Not okay', 'I feel sad', "I don't feel so good", 'depressed', 'sad', 'not good'], 'responses': ['Oh, I see. I hope everything will turn out well in the end. Anyway, what can I do for you?'], 'context_filter': 'question'}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", "Thank's a lot!"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure'], 'context_set': ''}, {'tag': 'closing', 'pat

In [3]:
# initializing lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer()
# Each list to create
words = []
classes = []
doc_X = []
doc_y = []
# Loop through all the intents
# tokenize each pattern and append tokens to words, the patterns and
# the associated tag to their associated list
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    
    # add the tag to the classes if it's not there already 
    if intent["tag"] not in classes:
        classes.append(intent["tag"])
# lemmatize all the words in the vocab and convert them to lowercase
# if the words don't appear in punctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
# sorting the vocab and classes in alphabetical order and taking the # set to ensure no duplicates occur
words = sorted(set(words))
classes = sorted(set(classes))
print(words)

["'m", "'s", 'a', 'about', 'ac', 'accountancy', 'address', 'am', 'an', 'and', 'applicant', 'application', 'applying', 'arch', 'archi', 'architecture', 'are', 'art', 'available', 'b', 'ba', 'bad', 'beced', 'beed', 'bio', 'biology', 'bpe', 'bsarchi', 'bsba', 'bsba-em', 'bsba-hrdm', 'bsce', 'bsche', 'bscs', 'bsed', 'bsee', 'bsem', 'bsge', 'bshm', 'bsie', 'bsit', 'bsme', 'bsmls', 'bspolsci', 'bsswk', 'by', 'bye', 'can', 'checklist', 'com', 'computing', 'contact', 'could', 'course', 'curiculim', 'curiculum', 'currently', 'day', 'dean', 'depressed', 'description', 'do', 'doe', 'education', 'email', 'eng', 'engineer', 'engineering', 'enginering', 'engl', 'enroll', 'enrolled', 'enrollment', 'entrance', 'entrep', 'exam', 'examination', 'existing', 'failed', 'fee', 'feel', 'feeling', 'finance', 'finman', 'for', 'freshies', 'freshman', 'funny', 'good', 'goodbye', 'grade', 'great', 'have', 'hello', 'helpful', 'hey', 'hi', 'hotline', 'how', 'i', 'if', 'in', 'incoming', 'industrial', 'information', 

In [4]:
# list for training data
training = []
out_empty = [0] * len(classes)
# creating the bag of words model
for idx, doc in enumerate(doc_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    # mark the index of class that the current pattern is associated
    # to
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1
    # add the one hot encoded BoW and associated classes to training 
    training.append([bow, output_row])
# shuffle the data and convert it to an array
random.shuffle(training)
training = np.array(training, dtype=object)
# split the features and target labels
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [5]:
# defining some parameters
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
epochs = 200
# the deep learning model
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_y, epochs=200, verbose=1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               25856     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 40)                2600      
Total params: 36,712
Trainable params: 36,712
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
7/7 [==============================] - 0s 2ms/step - loss: 3.5411 - accuracy: 0.0922
Epoch 2/200
7/7 

7/7 [==============================] - 0s 2ms/step - loss: 0.0912 - accuracy: 0.9631
Epoch 75/200
7/7 [==============================] - 0s 2ms/step - loss: 0.1779 - accuracy: 0.9539
Epoch 76/200
7/7 [==============================] - 0s 2ms/step - loss: 0.1661 - accuracy: 0.9585
Epoch 77/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0558 - accuracy: 0.9908
Epoch 78/200
7/7 [==============================] - 0s 1ms/step - loss: 0.1371 - accuracy: 0.9631
Epoch 79/200
7/7 [==============================] - 0s 2ms/step - loss: 0.1190 - accuracy: 0.9677
Epoch 80/200
7/7 [==============================] - 0s 1ms/step - loss: 0.1101 - accuracy: 0.9677
Epoch 81/200
7/7 [==============================] - 0s 1ms/step - loss: 0.1092 - accuracy: 0.9539
Epoch 82/200
7/7 [==============================] - 0s 2ms/step - loss: 0.1897 - accuracy: 0.9493
Epoch 83/200
7/7 [==============================] - 0s 2ms/step - loss: 0.1807 - accuracy: 0.9447
Epoch 84/200
7/7 [===============

7/7 [==============================] - 0s 2ms/step - loss: 0.3009 - accuracy: 0.9447
Epoch 158/200
7/7 [==============================] - 0s 1ms/step - loss: 0.1908 - accuracy: 0.9447
Epoch 159/200
7/7 [==============================] - 0s 1ms/step - loss: 0.1973 - accuracy: 0.9447
Epoch 160/200
7/7 [==============================] - 0s 1ms/step - loss: 0.1276 - accuracy: 0.9585
Epoch 161/200
7/7 [==============================] - 0s 2ms/step - loss: 0.1205 - accuracy: 0.9724
Epoch 162/200
7/7 [==============================] - 0s 1ms/step - loss: 0.2628 - accuracy: 0.9355
Epoch 163/200
7/7 [==============================] - 0s 2ms/step - loss: 0.1706 - accuracy: 0.9631
Epoch 164/200
7/7 [==============================] - 0s 2ms/step - loss: 0.2321 - accuracy: 0.9355
Epoch 165/200
7/7 [==============================] - 0s 2ms/step - loss: 0.1396 - accuracy: 0.9724
Epoch 166/200
7/7 [==============================] - 0s 2ms/step - loss: 0.0560 - accuracy: 0.9908
Epoch 167/200
7/7 [=====

In [6]:
def clean_text(text): 
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

def bag_of_words(text, vocab): 
    tokens = clean_text(text)
    bow = [0] * len(vocab)
    for w in tokens: 
        for idx, word in enumerate(vocab):
            if word == w: 
                bow[idx] = 1
    return np.array(bow)

def pred_class(text, vocab, labels): 
    bow = bag_of_words(text, vocab)
    result = model.predict(np.array([bow]))[0]
    thresh = 0.2
    y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]

    y_pred.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in y_pred:
        return_list.append(labels[r[0]])
    return return_list

def get_response(intents_list, intents_json): 
    tag = intents_list[0]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents: 
        if i["tag"] == tag:
            result = random.choice(i["responses"])
            break
    return result

In [ ]:
from flask import Flask, render_template, request, redirect, url_for 

app = Flask(__name__)
app.static_folder = 'static'

r = sr.Recognizer()

@app.route("/")
def home():
    return render_template("index.html")
    
@app.route("/get")
def get_bot_response():
    userText = request.args.get('msg')
    intents = pred_class(userText, words, classes)
    return str(get_response(intents, data))

@app.route('/result',methods = ['POST', 'GET'])
def result():
    if request.method == 'POST':
        result = request.form
        return render_template("result.html",result = result)
    
if __name__ == '__main__':
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Jul/2021 15:20:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 15:20:57] "GET /get?msg=hello HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 15:21:02] "GET /get?msg=how%20are%20you HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 15:21:06] "GET /get?msg=i%27m%20sad HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 15:21:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 15:21:29] "GET /get?msg=hello HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 15:21:31] "GET /get?msg=how%20are%20you HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 15:21:32] "GET /get?msg=sad HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 15:21:35] "GET /get?msg=payment HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 15:21:38] "GET /get?msg=pay HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 15:22:07] "GET /get?msg=yes HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 15:22:09] "GET /get?msg=no HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 15:22:13] "GET /get?msg=hello HTTP/1.1" 200 